## Use decision optimization
### Step 1: Import the docplex package

In [1]:
#pip install geopy

In [2]:
#pip install folium

In [3]:
#pip install cplex

In [4]:
import sys
import docplex.mp

### Step 3: Prepare the data
We need to collect the list of public libraries locations and keep their names, latitudes, and longitudes.

In [5]:
# Store longitude, latitude and street crossing name of each public library location.
class XPoint(object):
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __str__(self):
        return "P(%g_%g)" % (self.x, self.y)
    
class NamedPoint(XPoint):
    def __init__(self, name, x, y):
        XPoint.__init__(self, x, y)
        self.name = name
    def __str__(self):
        return self.name

#### Define how to compute the earth distance between 2 points
To easily compute distance between 2 points, we use the Python package geopy

In [6]:
try:
    import geopy.distance
except:
    if hasattr(sys, 'real_prefix'):
        # we are in a virtual env
        !pip install geopy
    else:
        !pip install --user geopy
        

In [7]:
# Simple distance compuation between 2 locations.
from geopy.distance import great_circle
 
def get_distance(p1, p2):
    return great_circle((p1.y, p1.x), (p2.y, p2.x)).miles

#### Declare the list of libraries
Parse the JSON file to get the list of libraries and store them as Python elements.

In [8]:
def build_libraries_from_url(url, name_pos, lat_long_pos):
    import requests
    import json

    r = requests.get(url)
    myjson = json.loads(r.text, parse_constant='utf-8')
    myjson = myjson['data']

    libraries = []
    k = 1
    for location in myjson:
        uname = location[name_pos]
        try:
            latitude = float(location[lat_long_pos][1])
            longitude = float(location[lat_long_pos][2])
        except TypeError:
            latitude = longitude = None
        try:
            name = str(uname)
        except:
            name = "???"
        name = "P_%s_%d" % (name, k)
        if latitude and longitude:
            cp = NamedPoint(name, longitude, latitude)
            libraries.append(cp)
            k += 1
    return libraries

In [9]:
libraries = build_libraries_from_url('https://data.cityofchicago.org/api/views/x8fc-8rcq/rows.json?accessType=DOWNLOAD',
                                   name_pos=10,
                                   lat_long_pos=16)

In [10]:
print("There are %d public libraries in Chicago" % (len(libraries)))

There are 81 public libraries in Chicago


#### Define number of shops to open
Create a constant that indicates how many coffee shops we would like to open.

In [11]:
nb_shops = 5
print("We would like to open %d coffee shops" % nb_shops)

We would like to open 5 coffee shops


#### Validate the data by displaying them
We will use the folium library to display a map with markers.

In [12]:
try:
    import folium
except:
    if hasattr(sys, 'real_prefix'):
        #we are in a virtual env.
        !pip install folium 
    else:
        !pip install folium

In [13]:
import folium
map_osm = folium.Map(location=[41.878, -87.629], zoom_start=11)
for library in libraries:
    lt = library.y
    lg = library.x
    folium.Marker([lt, lg]).add_to(map_osm)
map_osm

After running the above code, the data is displayed but it is impossible to determine where to ideally open the coffee shops by just looking at the map.

Let's set up DOcplex to write and solve an optimization model that will help us determine where to locate the coffee shops in an optimal way.

## Step 4: Set up the prescriptive model


In [14]:
from docplex.mp.environment import Environment
env = Environment()
env.print_information()

* system is: Darwin 64bit
* Python version 3.8.11, located at: /opt/anaconda3/bin/python
* docplex is present, version is 2.25.236
* CPLEX library is present, version is 22.1.0.0, located at: /opt/anaconda3/lib/python3.8/site-packages
* pandas is present, version is 1.3.4


#### Create the DOcplex model
The model contains all the business constraints and defines the objective.

In [15]:
from docplex.mp.model import Model

mdl = Model("coffee shops")

#### Define the decision variables

In [16]:
BIGNUM = 999999999

# Ensure unique points
libraries = set(libraries)
# For simplicity, let's consider that coffee shops candidate locations are the same as libraries locations.
# That is: any library location can also be selected as a coffee shop.
coffeeshop_locations = libraries

# Decision vars
# Binary vars indicating which coffee shop locations will be actually selected
coffeeshop_vars = mdl.binary_var_dict(coffeeshop_locations, name="is_coffeeshop")
#
# Binary vars representing the "assigned" libraries for each coffee shop
link_vars = mdl.binary_var_matrix(coffeeshop_locations, libraries, "link")

#### Express the business constraints
First constraint: if the distance is suspect, it needs to be excluded from the problem.

In [17]:
for c_loc in coffeeshop_locations:
    for b in libraries:
        if get_distance(c_loc, b) >= BIGNUM:
            mdl.add_constraint(link_vars[c_loc, b] == 0, "ct_forbid_{0!s}_{1!s}".format(c_loc, b))

Second constraint: each library must be linked to a coffee shop that is open.

In [18]:
mdl.add_constraints(link_vars[c_loc, b] <= coffeeshop_vars[c_loc]
                   for b in libraries
                   for c_loc in coffeeshop_locations)
mdl.print_information()

Model: coffee shops
 - number of variables: 6642
   - binary=6642, integer=0, continuous=0
 - number of constraints: 6561
   - linear=6561
 - parameters: defaults
 - objective: none
 - problem type is: MILP


Third constraint: each library is linked to exactly one coffee shop.

In [19]:
mdl.add_constraints(mdl.sum(link_vars[c_loc, b] for c_loc in coffeeshop_locations) == 1
                   for b in libraries)
mdl.print_information()

Model: coffee shops
 - number of variables: 6642
   - binary=6642, integer=0, continuous=0
 - number of constraints: 6642
   - linear=6642
 - parameters: defaults
 - objective: none
 - problem type is: MILP


Fourth constraint: there is a fixed number of coffee shops to open.

In [20]:
# Total nb of open coffee shops
mdl.add_constraint(mdl.sum(coffeeshop_vars[c_loc] for c_loc in coffeeshop_locations) == nb_shops)

# Print model information
mdl.print_information()

Model: coffee shops
 - number of variables: 6642
   - binary=6642, integer=0, continuous=0
 - number of constraints: 6643
   - linear=6643
 - parameters: defaults
 - objective: none
 - problem type is: MILP


#### Express the objective
The objective is to minimize the total distance from libraries to coffee shops so that a book reader always gets to our coffee shop easily.

In [21]:
# Minimize total distance from points to hubs
total_distance = mdl.sum(link_vars[c_loc, b] * get_distance(c_loc, b) for c_loc in coffeeshop_locations for b in libraries)
mdl.minimize(total_distance)

#### Solve with the Decision Optimization solve service
Solve the model.

In [22]:
print("# coffee shops locations = %d" % len(coffeeshop_locations))
print("# coffee shops           = %d" % nb_shops)

assert mdl.solve(), "!!! Solve of the model fails"

# coffee shops locations = 81
# coffee shops           = 5


DOcplexLimitsExceeded: **** Promotional version. Problem size limits (1000 vars, 1000 consts) exceeded, model has 6642 vars, 6643 consts, CPLEX code=1016